In [3]:
import cv2
import mediapipe as mp
import sys
import os
import logging

class QbitException(Exception):

    def __init__(self, error_message: Exception, error_detail: sys):
        super().__init__(error_message)
        self.error_message = QbitException.get_detailed_error_message(error_message=error_message,
                                                                         error_detail=error_detail
                                                                         )

    @staticmethod
    def get_detailed_error_message(error_message: Exception, error_detail: sys) -> str:
        """
        error_message: Exception object
        error_detail: object of sys module
        """
        _, _, exec_tb = error_detail.exc_info()
        try_block_line_number = exec_tb.tb_lineno
        file_name = exec_tb.tb_frame.f_code.co_filename
        error_message = f"Error occurred in script: [ {file_name} ] at line number: [{try_block_line_number}] error " \
                        f"message: [{error_message}] "

        return error_message

    def __str__(self):
        return self.error_message

    def __repr__(self) -> str:
        return str(QbitException.__name__)

    
LOG_DIR = "logs"

LOG_FILE_NAME = "loger_file.log"

os.makedirs(LOG_DIR, exist_ok=True)

LOG_FILE_PATH = os.path.join(LOG_DIR, LOG_FILE_NAME)

logging.basicConfig(filename=LOG_FILE_PATH,
                    filemode="w",
                    format='[%(asctime)s] \t%(levelname)s \t%(lineno)d \t%(filename)s \t%(funcName)s() \t%(message)s',
                    level=logging.INFO
                    )
    

class DistanceEstimator:
    def __init__(self):
        self.mp_drawing = mp.solutions.drawing_utils
        self.mp_face = mp.solutions.face_detection.FaceDetection(model_selection=1, min_detection_confidence=0.5)
        self.known_distance = 69.0
        self.known_width = 14.0
        self.a = []
        self.width = None
        self.height = None

    def focal_length_finder(self, real_width, width_in_rf_image):
        focal_length = (width_in_rf_image * self.known_distance) / real_width
        return focal_length

    def obj_data(self, img):
        obj_width = 0
        image_input = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        results = self.mp_face.process(image_input)
        try:
            if not results.detections:
                ValueError("No Faces")
            else:
                for detection in results.detections:
                    bbox = detection.location_data.relative_bounding_box
                    x, y, w, h = int(bbox.xmin * self.width), int(bbox.ymin * self.height), int(bbox.width * self.width), int(
                        bbox.height * self.height)
                    cv2.rectangle(img, (x, y), (x + w, y + h), (0, 0, 255), 2)
                    self.a.append([x, y])
                    obj_width = w
                return obj_width
        except Exception as e:
            raise QbitException(e, sys)
        

    def distance_finder(self, focal_length, obj_width_in_frame):
        distance = (self.known_width * focal_length) / obj_width_in_frame
        return distance

    def process_image(self, img_path):
        img = cv2.imread(img_path)
        try:
            if img is None:
                ValueError('No frames available for processing.') 

            self.width = img.shape[1]
            self.height = img.shape[0]

            ref_image_obj_width = self.obj_data(img)
            if ref_image_obj_width == 0:
                ValueError('No object detected') 
                return

            focal_length_found = self.focal_length_finder(self.known_width, ref_image_obj_width)
            logging.info(f"focal length found {focal_length_found}")

            obj_width_in_frame = self.obj_data(img)
            if obj_width_in_frame:
                distance = self.distance_finder(focal_length_found, obj_width_in_frame)
                for i in self.a:
                    x1 = i[0]
                    y1 = i[1]
                cv2.putText(img, f"Distance: {round(distance, 2)} CM", (x1, y1), cv2.FONT_HERSHEY_PLAIN, 2, (255, 0, 0), 2)
            else:
                logging.info("no faces detected")

            # cv2.imshow("Image with Distance Annotation", img)
            the_value = {'distance_to_camera':(round(distance, 2))}
            return the_value
        except Exception as e:
            raise QbitException(e, sys)
   


distance_estimator = DistanceEstimator()
image_path = '1.png'
distance_estimator.process_image(image_path)


libEGL warning: MESA-LOADER: failed to open radeonsi: /usr/lib/dri/radeonsi_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)

libEGL warning: MESA-LOADER: failed to open swrast: /usr/lib/dri/swrast_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)

libEGL warning: MESA-LOADER: failed to open radeonsi: /usr/lib/dri/radeonsi_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)

libEGL warning: MESA-LOADER: failed to open swrast: /usr/lib/dri/swrast_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)

W0000 00:00:1717423788.822521   17095 inference_feedback_manager.cc:114] Feedback manage

{'distance_to_camera': 67.55}